## SVM (Support vector machines)

**SVM** are one of the most popular ML algorithm for classification task. In this notebook we will implement a SVM classifier from default and compare the results with Sklearn. **SVM** produce non linear boundaries by constructing a linear boundary in a large, transformed version of the feature space.

**SVM** solves the following primal problem:

$\text{min}_{\beta,\beta_0} \frac{1}{2} || \beta ||^2 + C \sum_{i=1}^{N} \xi_i$

subject to $\xi_i \ge 0$, $y_i(x_i^T\beta+\beta_0) \ge 1 - \xi_i$ $\forall i$.

Dual of the problem is

$\text{min}_{\alpha} \frac{1}{2} \alpha^T Q \alpha - e^T \alpha$

subject to $y^T\alpha = 0$, $0 \le \alpha_i \le C, i = 1...n$, where $e$ is the vector of all ones. And $Q$ is a $n \times n$ matrix $Q_{ij}=y_iy_jK(x_i,x_j)$, where $K(x_i,x_j) = \phi(x_i)^T\phi(x_j)$.

Once optimal values of $\alpha_i$ are found decision function is given as 

$\sum_{i \in SV} y_i \alpha_i K(x_i, x) + b$. Below we will use cvxopt to solve the QP for **SVM**


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.svm import SVC
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [2]:
#Make a classification dataset
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=100, 
                           n_features=5, 
                           random_state=0)
y = 2*y-1
print(X.shape, y.shape)

(100, 5) (100,)


In [3]:
#Divide data in train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)
print(X_train.shape, X_test.shape)

(90, 5) (10, 5)


In [4]:
from sklearn import svm
clf = svm.SVC(kernel='linear', C=1)
clf.fit(X_train, y_train)
print(f"Train score {clf.score(X_train, y_train) : .3}")
print(f"Test score {clf.score(X_test, y_test) : .3}")

print(f"Coef= {clf.coef_}")
print(f"Intercept = {clf.intercept_}")

Train score  0.933
Test score  0.9
Coef= [[0.97281344 0.68848874 0.09705952 0.79525107 0.1183482 ]]
Intercept = [0.57480184]


#### Now lets implement SVM from scratch

**cvsopt** API is 

$\text{min}\frac{1}{2}x^TPx + q^Tx$  s.t. $Gx \le h$  $Ax = b$

Reformulating **SVM** equation in the above format we have
1. $P=Q$
2. $q=-e$
3. $A=y$
4. $b=0$
5. $G=diag(1)$ stacked over $diag(-1)$
6. $h$ is a vector of $C$ stacked over a vector of 0s.

In [5]:
from sklearn.metrics import accuracy_score
class SVM():
    """
        Implements Linear SVM for classification
    """
    def __init__(self,C=1):
        self.C_ = C
        
    def fit(self, X, y):
        #Lets compute the Q matrix
        (n,m) = X.shape
        y = y.reshape(y.shape[0],1)
        y = y.astype('float')
        yx = y * X
        Q = np.dot(yx , yx.T)
        
        #set e
        e = -np.ones((n,1))
        
        #Convert to cvxopt_matrix
        P = cvxopt_matrix(Q)
        q = cvxopt_matrix(e)
        A = cvxopt_matrix(y.reshape(1,-1))
        G = cvxopt_matrix(np.vstack((np.eye(n),np.eye(n)*-1)))
        h = cvxopt_matrix(np.hstack((np.ones(n) * self.C_, np.zeros(n))))
        b = cvxopt_matrix(np.zeros(1))
        
        
        cvxopt_solvers.options['show_progress'] = True
        cvxopt_solvers.options['abstol'] = 1e-3
        cvxopt_solvers.options['reltol'] = 1e-3
        cvxopt_solvers.options['feastol'] = 1e-3

        #Run solver
        sol = cvxopt_solvers.qp(P, q, G, h, A, b)
        alphas = np.array(sol['x'])
        beta = ((y * alphas).T @ X).reshape(-1,1)
        
        #many alphas will be zero
        #Alphas which are non zero make the support vector
        S = (alphas > 1e-4).flatten()
        
        #compute beta_0
        beta_0 = y[S] - np.dot(X[S], beta)
        
        self.coef_ = beta.T
        #Take the beta computed by first Support vector
        self.intercept_ = beta_0[0]
        
    def predict(self, X):
        return np.sign(self.intercept_ + np.matmul(X,self.coef_.reshape(-1,1)))

    def score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)

In [6]:
clf = SVM()
clf.fit(X_train, y_train)

print(f"Coef= {clf.coef_}")
print(f"Intercept = {clf.intercept_}")

print(f"Train score {clf.score(X_train, y_train) : .3}")
print(f"Test score {clf.score(X_test, y_test) : .3}")

     pcost       dcost       gap    pres   dres
 0: -3.7359e+01 -2.3730e+02  1e+03  3e+00  6e-15
 1: -2.3489e+01 -1.5904e+02  3e+02  4e-01  4e-15
 2: -1.6704e+01 -6.2924e+01  7e+01  9e-02  6e-15
 3: -1.6005e+01 -2.6826e+01  1e+01  2e-02  4e-15
 4: -1.7776e+01 -2.1906e+01  5e+00  5e-03  3e-15
 5: -1.8372e+01 -2.0651e+01  3e+00  2e-03  3e-15
 6: -1.8703e+01 -1.9743e+01  1e+00  6e-04  3e-15
 7: -1.8942e+01 -1.9363e+01  4e-01  1e-04  3e-15
 8: -1.9054e+01 -1.9192e+01  1e-01  3e-05  3e-15
 9: -1.9083e+01 -1.9149e+01  7e-02  1e-05  3e-15
10: -1.9110e+01 -1.9116e+01  5e-03  6e-16  3e-15
Optimal solution found.
Coef= [[0.97293264 0.68859025 0.09684185 0.79533827 0.11838178]]
Intercept = [0.5745351]
Train score  0.933
Test score  0.9


#### Our results match very closely with sklearns svm implementation!!!

## Kernel SVM

In [7]:
#Lets apply Kernel SVM to the data and see if it changes results
clf = svm.SVC(kernel='rbf', C=1)
clf.fit(X_train, y_train)
print(f"Train score {clf.score(X_train, y_train) : .3}")
print(f"Test score {clf.score(X_test, y_test) : .3}")

Train score  0.944
Test score  0.9


Using rbf kernel doesn't seem to help much in this case.